In [1]:
####### Import Necessary Libraries #######
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


####### Data Loading && Preparation #######
import nltk 
nltk.download('punkt')
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, PreTrainedModel, PretrainedConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding


####### Modelling #######
import torch
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split



#######Evaluation & Result #######
from sklearn.metrics import classification_report, confusion_matrix,  accuracy_score, precision_recall_fscore_support
import itertools


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Loading

In [3]:
BLOOM_DATASET = '../../Dataset/learning_outcomes.csv'
PAIR_DATASET = '../../Dataset/cpmk_subcpmk_pairs.csv'

In [4]:
bloom_df = pd.read_csv(BLOOM_DATASET)
pair_df = pd.read_csv(PAIR_DATASET)

print(bloom_df)
pair_df

                                       Learning_Outcome     Jenis Level
0     Mampu merencanakan, menyelesaikan, dan mengeva...      CPMK    C6
1     Mampu merencanakan, menyelesaikan, dan mengeva...      CPMK    C6
2     Mampu merencanakan, menyelesaikan, dan mengeva...      CPMK    C6
3                           Mampu merancang gardu induk      CPMK    C6
4                           Mampu merancang gardu induk      CPMK    C6
...                                                 ...       ...   ...
2571  Mahasiswa dapat memahami dan mempraktekkan iba...  Sub-CPMK    P3
2572  Mampu menunjukan pelaksanaan penegakan hukum &...  Sub-CPMK    A5
2573  Mahasiswa mampu menunjukkan implementasi nilai...  Sub-CPMK    A5
2574  Mampu bertindak mengimplementasikan makna Sila...  Sub-CPMK    A5
2575  Mampu bertindak mengimplementasikan Sila Ke-4 ...  Sub-CPMK    A5

[2576 rows x 3 columns]


,cpmk,level_cpmk,subcpmk,level_subcpmk,keselarasan
0,"Mampu merencanakan, menyelesaikan, dan mengeva...",C6,Merencanakan aplikasi menggunakan prinsip dasa...,C6,True
1,"Mampu merencanakan, menyelesaikan, dan mengeva...",C6,Menyelesaikan logic functions and gates,C2,True
2,"Mampu merencanakan, menyelesaikan, dan mengeva...",C6,Mengevaluasi boolean algebra dan combinational...,C5,True
3,Mampu merancang gardu induk,C6,Mampu merancang instalasi listrik gardu induk,C6,True
4,Mampu merancang gardu induk,C6,Mampu merancang sistem pengetanahan gardu induk,C6,True
...,...,...,...,...,...
1283,"Mahasiswa mampu mempraktekkan haji, umroh dan ...",P3,Mahasiswa dapat memahami dan mempraktekkan iba...,P3,True
1284,"Mampu menganalisis masalah kontekstual PKn, me...",C4,Mampu menunjukan pelaksanaan penegakan hukum &...,A5,False
1285,Mahasiswa mampu menafsirkan dan menerapkan nil...,C3,Mahasiswa mampu menunjukkan implementasi nilai...,A5,False
1286,"Mampu menunjukan nilai ketuhanan, nilai kemanu...",A5,Mampu bertindak mengimplementasikan makna Sila...,A5,True


# Data Preparation & Modelling - Bloom

## Data Preparation

In [5]:
# Basic checks & cleaning
required_cols = ["Learning_Outcome", "Level"]
if not all(c in bloom_df.columns for c in required_cols):
    raise ValueError(f"CSV must contain columns: {required_cols}. Found: {list(bloom_df.columns)}")

bloom_df = bloom_df.dropna(subset=["Learning_Outcome", "Level"]).reset_index(drop=True)
bloom_df["Learning_Outcome"] = bloom_df["Learning_Outcome"].astype(str).str.strip()
bloom_df["Level"] = bloom_df["Level"].astype(str).str.strip().str.upper()

In [6]:
# Create label mapping
labels = sorted(bloom_df["Level"].unique().tolist(), key=lambda x: (x[0], int(x[1:]) if x[1:].isdigit() else x))
label2id = {lab:i for i, lab in enumerate(labels)}
id2label = {i:lab for lab,i in label2id.items()}

bloom_df["label_id"] = bloom_df["Level"].map(label2id)

In [7]:
# Train / val split (stratify by label)
train_df, val_df = train_test_split(bloom_df, test_size=0.1, random_state=42, stratify=bloom_df["label_id"])
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
datasets = DatasetDict({"train": train_ds, "validation": val_ds})


In [8]:
# Tokenizer and preprocessing
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def preprocess_fn(batch):
    enc = tokenizer(batch["Learning_Outcome"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"] = batch["label_id"]
    return enc

datasets = datasets.map(preprocess_fn, batched=True, remove_columns=datasets["train"].column_names)


NameError: name 'MODEL_NAME' is not defined

## Modelling 

In [9]:
INDO_BERT_BASE = "indobenchmark/indobert-large-p2"   


### Indo-Bert-Base

In [10]:
OUTPUT_DIR = "Classification-result/indobert_single_head_out"
SEED = 42
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
MAX_LEN = 512

In [11]:
# Model
num_labels = len(label2id)
model = AutoModelForSequenceClassification.from_pretrained(INDO_BERT_BASE , num_labels=num_labels)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   1%|          | 10.5M/1.34G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [12]:
def compute_metrics(eval_pred):
    preds_logits, labels = eval_pred
    preds = np.argmax(preds_logits, axis=-1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    class_report = classification_report(labels, preds, target_names=labels)
    print("Classification Report:\n", class_report)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}
    

In [13]:

# Training args
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    logging_steps=50,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
)



In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()

In [ ]:
# Eval final
metrics = trainer.evaluate()
print("Final evaluation metrics:", metrics)

# Save model, tokenizer, and label maps
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
pd.to_pickle(label2id, os.path.join(OUTPUT_DIR, "label2id.pkl"))
pd.to_pickle(id2label, os.path.join(OUTPUT_DIR, "id2label.pkl"))
print("Model & tokenizer saved to", OUTPUT_DIR)